# **Art Classifier**

## **Non Structured Data**

This project has been done by:

|Name                    |Email                              |
|------------------------|-----------------------------------|
|Jorge Ayuso Martínez    |jorgeayusomartinez@alu.comillas.edu|
|Carlota Monedero Herranz|carlotamoh@alu.comillas.edu        |
|José Manuel Vega Gradit |josemanuel.vega@alu.comillas.edu   |

First of all, let's load the required libraries in order to run the code:

In [9]:
import os
import numpy as np

import tensorflow.keras as keras
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns

Once we have done so, let's mount Google Drive:

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now let's see how our data is structured:

In [5]:
# Root folder
base_dir = "/content/drive/MyDrive/images/images"

In [ ]:
# Train folder
train_dir = ""

# Validation folder
validation_dir = ""

# Test folder
test_dir = ""

In [ ]:
# See folder names inside root folder
for path in os.walk(base_dir):
    for folder in path[1]:
        print(folder)

Let's also see how many images there are for each class in the training, validation and test set.

In [ ]:
# Number of classes
n_classes = len(os.listdir(train_dir))
print(f"Number of classes: {n_classes}")

# Get existing classes
classes = os.listdir(train_dir)
print("Existing classes:\n")
classes

In [ ]:
# Training
print("Number of images per class in Training set:")
print("="*50)
for cl in classes:
    n_images = len(os.listdir(os.path.join(train_dir, cl)))
    print(f"{cl}: {n_images}")

In [ ]:
# Validation
print("Number of images per class in Validation set:")
print("="*50)
for cl in classes:
    n_images = len(os.listdir(os.path.join(validation_dir, cl)))
    print(f"{cl}: {n_images}")

In [ ]:
# Test
print("Number of images per class in Test set:")
print("="*50)
for cl in classes:
    n_images = len(os.listdir(os.path.join(test_dir, cl)))
    print(f"{cl}: {n_images}")

## **1. Base model**

The first model we will create is a simple CNN. This will help us to have a general idea of how a very simple model would perform classifying art images, and try to improve this model by, for instance: 

* Upsampling the training dataset using Data Augmentation.
* Using regularization techniques such as $𝓛_1$, $𝓛_2$ or dropout.
* Incrementing the number of parameters of the CNN.
* Using pre-trained models (Transfer-Learning and Fine-Tuning).

### 1.1. Model structure

Let's first create the model structure:

In [ ]:
model = models.Sequential()
# 1st Convolution Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)
                        )
)
# 1st Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 2nd Convolution Layer
model.add(layers.Conv2D(64
                        , (3, 3)
                        , activation='relu'
                        )
)
# 2nd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 3rd Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 3rd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 4th Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 4th Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [ ]:
model.summary()

We'll use Adam as our optimizer since it is the most popular optimizer right now, as well as versatile (i.e., it can be used in multiple contexts).

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=1e-4),
    metrics=['acc']
)

### 1.2. Data preprocessing

A critical step when creating these kind of models is how the input data is preprocessed. These involves:

* Normalize the input by dividing each pixel by its maximum value (i.e, 255).
* Define the input size, which affects to the final model.
* Batch size: this is the number of images in each batch.

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # We use raw to get predicted probabilities
        class_mode='raw'
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='raw'
        )

Now let's take a look at the output of one of these generators (for instance, the training one):

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

*We can appreciate that...*

### 1.3. Training

Let's train the model:

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50
      )

And once the model has been trained, we will save it:

In [ ]:
# Create directory where to save the models created
models_dir = "/content/drive/MyDrive/models"
os.makedirs(models_dir, exist_ok=True)

# Directory where to save the model

# Save model
base_model_dir = os.path.join(models_dir, "model_v0.h5")
model.save(base_model_dir)

### 1.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

*Comments about how those metrics have evolved...*

## **2. Including Dropout**

*Explain dropout, include reference to original paper*

### 2.1. Model structure

Let's first create the model structure:

In [ ]:
model = models.Sequential()
# 1st Convolution Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)
                        )
)
# 1st Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 2nd Convolution Layer
model.add(layers.Conv2D(64, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 2nd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 3rd Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 3rd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 4th Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 4th Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [ ]:
model.summary()

We'll use Adam as our optimizer, as well as we have done previously due to the reasons already mentioned.

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=1e-4),
    metrics=['acc']
)

### 2.2. Data preprocessing

We'll apply the same preprocess steps we have performed for the base model.

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # We use raw to get predicted probabilities
        class_mode='raw'
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='raw'
        )

### 2.3. Training

Let's train the model:

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50
      )

And once the model has been trained, we will save it:

In [ ]:
# Create directory where to save the models created
models_dir = "/content/drive/MyDrive/models"
os.makedirs(models_dir, exist_ok=True)

# Directory where to save the model

# Save model
base_model_dir = os.path.join(models_dir, "model_v1.h5")
model.save(base_model_dir)

### 2.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

*Comments about how those metrics have evolved... and compare them with prior results.*

## **3. Including Data Augmentation**

*Explain Data Augmentation, reference some website or paper which talks about this technique.*

### 3.1. Model structure

Let's first create the model structure:

In [ ]:
model = models.Sequential()
# 1st Convolution Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)
                        )
)
# 1st Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 2nd Convolution Layer
model.add(layers.Conv2D(64, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 2nd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 3rd Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 3rd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 4th Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 4th Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [ ]:
model.summary()

We'll use Adam as our optimizer, as well as we have done previously due to the reasons already mentioned.

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=1e-4),
    metrics=['acc']
)

### 3.2. Data preprocessing

In this case, we will include the Data Augmentation step to the model preprocessing step...

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    )

# The data augmentation must not be used for the test set!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # We use raw to get predicted probabilities
        class_mode='raw'
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='raw'
        )

### 3.3. Training

Let's train the model:

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50
      )

And once the model has been trained, we will save it:

In [ ]:
# Create directory where to save the models created
models_dir = "/content/drive/MyDrive/models"
os.makedirs(models_dir, exist_ok=True)

# Directory where to save the model

# Save model
base_model_dir = os.path.join(models_dir, "model_v2.h5")
model.save(base_model_dir)

### 3.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

*Comments about how those metrics have evolved... and compare those with prior results*

## **4. Model from scratch with best combination of hyperparameters found**

### 4.1. Model structure

Let's first create the model structure:

In [ ]:
model = models.Sequential()
# 1st Convolution Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)
                        )
)
# 1st Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 2nd Convolution Layer
model.add(layers.Conv2D(64, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 2nd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 3rd Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 3rd Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
# 4th Convolution Layer
model.add(layers.Conv2D(128, 
                        (3, 3), 
                        activation='relu'
                        )
)
# 4th Pooling Layer
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Once the structure of the base model has been defined, let's see exactly how many parameters it has in order to have a better idea of how flexible this model is:

In [ ]:
model.summary()

We'll use Adam as our optimizer, as well as we have done previouly due to the reasons already mentioned.

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=1e-4),
    metrics=['acc']
)

### 4.2. Data preprocessing

We'll apply the same preprocess steps we have performed for the base model.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# The data augmentation must not be used for the test set!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # We use raw to get predicted probabilities
        class_mode='raw'
        )

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='raw'
        )

### 4.3. Training

Let's train the model:

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50
      )

And once the model has been trained, we will save it:

In [ ]:
# Create directory where to save the models created
models_dir = "/content/drive/MyDrive/models"
os.makedirs(models_dir, exist_ok=True)

# Directory where to save the model

# Save model
base_model_dir = os.path.join(models_dir, "model_v3.h5")
model.save(base_model_dir)

### 4.4. Validation

Let's plot how the loss and the accuracy from both training and validations sets have evolved during the training process. 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

*Comments about how those metrics have evolved...*

## **5. Fine-tuning**

We will use a pretrained model in order to compare its results with the results obtained with the models trained from scratch we already trained. Specifically, we will train a pretrained model found in Hugging Face which has 

First of all let's install the required libraries needed to run the pretrained model.

In [ ]:
!pip install ultralyticsplus==0.0.24 ultralytics==8.0.23

In [8]:
from ultralyticsplus import YOLO, postprocess_classify_output

# load model
model = YOLO('keremberke/yolov8m-painting-classification')

*Useful links for this model:*

[Hugging Face model link](https://huggingface.co/keremberke/yolov8m-painting-classification)

[Awesome Yolov8 models website](https://yolov8.xyz/#/?id=classification-models)

[Yolov8 GitHub page](https://github.com/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb)

[Ultralytics website](https://docs.ultralytics.com/python/)